In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)


/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "housematerial"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024

    # Use smaller anchors because our image and objects are small
    #RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 50
    # TRAIN_ROIS_PER_IMAGE = 20

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [3]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [4]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  /home/jupyter/Mask_RCNN/logs/housematerial20210415T2000/mask_rcnn_housematerial_0040.h5
Re-starting from epoch 40


In [ ]:
filenames=[]
original_images=[]
#folder = "/home/jupyter/tutorials/storage/detect/downloads_10"
folder = "/home/jupyter/Mask_RCNN/dataset/detect"
for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,filename))
    if img is not None:
        original_images.append(img)
        filenames.append(filename)

In [10]:

filenames

['280.jpg',
 '1612.jpg',
 '2106.jpg',
 '2622.jpg',
 '2123.jpg',
 '2376.jpg',
 '734.jpg',
 '1981.jpg',
 '912.jpg',
 '546.jpg',
 '2283.jpg',
 '2512.jpg',
 '553.jpg',
 '3014.jpg',
 '483.jpg',
 '3094.jpg',
 '3429.jpg',
 '3339.jpg',
 '192.jpg',
 '1908.jpg',
 '846.jpg',
 '2198.jpg',
 '1749.jpg',
 '1253.jpg',
 '1075.jpg',
 '3276.jpg',
 '2762.jpg',
 '2777.jpg',
 '3141.jpg',
 '2929.jpg',
 '1412.jpg',
 '2249.jpg',
 '3232.jpg',
 '3201.jpg',
 '1135.jpg',
 '3425.jpg',
 '2282.jpg',
 '2846.jpg',
 '1715.jpg',
 '3025.jpg',
 '2722.jpg',
 '988.jpg',
 '2378.jpg',
 '840.jpg',
 '1940.jpg',
 '86.jpg',
 '3207.jpg',
 '1935.jpg',
 '610.jpg',
 '2962.jpg',
 '3140.jpg',
 '2421.jpg',
 '1529.jpg',
 '3191.jpg',
 '2525.jpg',
 '491.jpg',
 '3242.jpg',
 '302.jpg',
 '960.jpg',
 '2400.jpg',
 '1587.jpg',
 '213.jpg',
 '3245.jpg',
 '2633.jpg',
 '1027.jpg',
 '1090.jpg',
 '3258.jpg',
 '1450.jpg',
 '3235.jpg',
 '3334.jpg',
 '3050.jpg',
 '2474.jpg',
 '1206.jpg',
 '1936.jpg',
 '2199.jpg',
 '2107.jpg',
 '991.jpg',
 '2485.jpg',
 '30

In [12]:
#import pandas as pd
#sta = pd.read_excel('statis.xlsx')
#sta['filename'] = filenames
#sta

,Unnamed: 0,1,2,3,4,filename
0,0,8,0,0,0,280.jpg
1,1,5,0,0,1,1612.jpg
2,2,5,0,0,0,2106.jpg
3,3,9,1,0,0,2622.jpg
4,4,8,0,0,0,2123.jpg
...,...,...,...,...,...,...
2378,2378,8,0,1,0,43.jpg
2379,2379,8,0,0,0,3153.jpg
2380,2380,12,0,1,0,811.jpg
2381,2381,3,0,0,0,2031.jpg


In [18]:
results = []
all_results = []
i = 1
for image in original_images:
    result = model.detect([image], verbose=0)
    r = result[0]
    results.append(r['class_ids'])
    all_results.append(r)
    print(i)
    i = i+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [19]:
statis = []
for i in results:
    dict = {}
    dict[1] = 0
    dict[2] = 0
    dict[3] = 0
    dict[4] = 0
    for key in i:
        dict[key] = dict.get(key, 0) + 1
    statis.append(dict)
statis
import pandas as pd
dt_statis = pd.DataFrame(statis)
dt_statis

,1,2,3,4
0,8,0,0,0
1,5,0,0,1
2,5,0,0,0
3,9,1,0,0
4,8,0,0,0
...,...,...,...,...
2378,8,0,1,0
2379,8,0,0,0
2380,12,0,1,0
2381,3,0,0,0


In [20]:
# dt_statis.to_excel('statis.xlsx')
dt_statis['filename'] = filenames

In [ ]:
dt_statis.to_excel('statis.xlsx')
dt_statis

In [23]:
dt_statis

,1,2,3,4,filename
0,8,0,0,0,280.jpg
1,5,0,0,1,1612.jpg
2,5,0,0,0,2106.jpg
3,9,1,0,0,2622.jpg
4,8,0,0,0,2123.jpg
...,...,...,...,...,...
2378,8,0,1,0,43.jpg
2379,8,0,0,0,3153.jpg
2380,12,0,1,0,811.jpg
2381,3,0,0,0,2031.jpg


In [22]:
import pandas as pd
statis = pd.read_excel('statis.xlsx')
origin = pd.read_excel('fitered_final.xlsx')
statis['filenames'] = data['filenames'].map(lambda x: x.rstrip('.jpg'))
origin.merge(dt_statis, left_on='original_number', right_on='filenames')

,building_lat,building_lon,original_number
0,51.510784,-0.091227,0
1,51.511627,-0.092403,1
2,51.514328,-0.091670,5
3,51.513775,-0.089974,6
4,51.513062,-0.087635,7
...,...,...,...
2381,51.515301,-0.096042,3454
2382,51.516991,-0.091228,3455
2383,51.515282,-0.095988,3456
2384,51.517033,-0.096512,3457


In [ ]:
origin.merge(dt_statis, left_on='original_number', right_on='filenames')

In [ ]:
j = 1
for resul in all_results:
    if resul is not None:
        resul['rois'] = resul['rois'].tolist()
        resul['class_ids'] = resul['class_ids'].tolist()
        resul['scores'] = resul['scores'].tolist()
        resul['masks'] = resul['masks'].tolist()
    print(j)
    j = j+1
    
import json

with open('data1.json', 'w') as fp:
    json.dump(all_results, fp)